# **Computation of PPP Metrics for Ensemble Runs**

**Important metrics:**

- SH sea ice extent/volume
- regional sea ice extent/volume
- regional NPP, PCO2, SSS, SST
- gridpoint-level fields for sea ice concentration, SST, SSS, NPP, PCO2
- sea ice edge position (latitude at which sea ice concentration drops below 15%)

In [1]:
import os
import warnings
warnings.filterwarnings("ignore", message="invalid value encountered in true_divide")
warnings.filterwarnings("ignore", message="Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range")
warnings.filterwarnings("ignore", message="invalid value encountered in reduce")

import xarray as xr
import numpy as np
import math
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
import hmei.processing as pcs
import hmei.visualization as vis

In [3]:
rootdir = '/local/projects/so_predict/esm2m_froelicher/'
writedir = '/home/bbuchovecky/storage/so_predict_derived/'

reg_masks = xr.open_dataset(writedir+'regional_global_masks.nc')

In [106]:
pcs.dir_inspect(writedir+'CTRL/'+'CN_INV')

['CN_raw',
 'cn_inv_ctrl_global_anomaly.nc',
 'cn_inv_ctrl_global_climatology.nc',
 'cn_inv_ctrl_global_variance.nc',
 'cn_inv_ctrl_so_anomaly.nc',
 'cn_inv_ctrl_so_climatology.nc',
 'cn_inv_ctrl_so_variance.nc',
 'cn_inv_global_annual_mean.nc',
 'cn_inv_global_monthly_anom.nc',
 'cn_inv_global_monthly_mean.nc',
 'cn_inv_global_monthly_var.nc',
 'cn_inv_so_annual_mean.nc',
 'cn_inv_so_monthly_anom.nc',
 'cn_inv_so_monthly_mean.nc',
 'cn_inv_so_monthly_var.nc',
 'ocean.static.nc']

In [56]:
pcs.dir_inspect(rootdir+'MLD_ENSEMBLE')

['ice_month.static.nc',
 'mld_ENS01_ensmn_0170_0179.nc',
 'mld_ENS01_neg01_0170_0179.nc',
 'mld_ENS01_neg02_0170_0179.nc',
 'mld_ENS01_neg03_0170_0179.nc',
 'mld_ENS01_neg04_0170_0179.nc',
 'mld_ENS01_neg05_0170_0179.nc',
 'mld_ENS01_neg06_0170_0179.nc',
 'mld_ENS01_neg07_0170_0179.nc',
 'mld_ENS01_neg08_0170_0179.nc',
 'mld_ENS01_neg09_0170_0179.nc',
 'mld_ENS01_neg10_0170_0179.nc',
 'mld_ENS01_neg11_0170_0179.nc',
 'mld_ENS01_neg12_0170_0179.nc',
 'mld_ENS01_neg13_0170_0179.nc',
 'mld_ENS01_neg14_0170_0179.nc',
 'mld_ENS01_neg15_0170_0179.nc',
 'mld_ENS01_neg16_0170_0179.nc',
 'mld_ENS01_neg17_0170_0179.nc',
 'mld_ENS01_neg18_0170_0179.nc',
 'mld_ENS01_neg19_0170_0179.nc',
 'mld_ENS01_neg20_0170_0179.nc',
 'mld_ENS01_pos01_0170_0179.nc',
 'mld_ENS01_pos02_0170_0179.nc',
 'mld_ENS01_pos03_0170_0179.nc',
 'mld_ENS01_pos04_0170_0179.nc',
 'mld_ENS01_pos05_0170_0179.nc',
 'mld_ENS01_pos06_0170_0179.nc',
 'mld_ENS01_pos07_0170_0179.nc',
 'mld_ENS01_pos08_0170_0179.nc',
 'mld_ENS01_pos09_0

In [36]:
subdir = 'NPP_ENSEMBLE/'
xr.open_dataset(rootdir+subdir+'NPP_ENS01_neg01_0170_0179.nc')

<xarray.Dataset>
Dimensions:        (TIME: 120, XT_OCEAN: 360, YT_OCEAN: 200, bnds: 2)
Coordinates:
  * XT_OCEAN       (XT_OCEAN) float64 -279.5 -278.5 -277.5 ... 77.5 78.5 79.5
  * YT_OCEAN       (YT_OCEAN) float64 -81.5 -80.5 -79.5 -78.5 ... 87.5 88.5 89.5
  * TIME           (TIME) object 0170-01-16 12:00:00 ... 0179-12-16 12:00:00
Dimensions without coordinates: bnds
Data variables:
    YT_OCEAN_bnds  (YT_OCEAN, bnds) float64 ...
    TIME_bnds      (TIME, bnds) object ...
    NPP            (TIME, YT_OCEAN, XT_OCEAN) float32 ...
Attributes:
    history:                   Sun Dec 22 15:32:33 2019: ncks -4 -L 1 -a -O d...
    Conventions:               CF-1.0
    nco_openmp_thread_number:  1
    NCO:                       netCDF Operators version 4.7.5 (Homepage = htt...

In [34]:
coords = list(x.coords)
if (coords.count('XT') == 1) and (coords.count('YT') == 1) and (coords.count('TIME') == 1):
    print('a')

if (coords.count('XT_OCEAN') == 1) and (coords.count('YT_OCEAN') == 1) and (coords.count('TIME') == 1):
    print('b')

In [98]:
subdir = 'SST_ENSEMBLE/'
x = xr.open_dataset(rootdir+'SST_ENSEMBLE/sst_ENS01_neg01_0170_0179.nc')
y = xr.open_dataset(rootdir+'SST_ENSEMBLE/sst_ENS01_pos01_0170_0179.nc')


In [102]:
## combine 
z = xr.concat([x,y], 'nEns')
# z = z.assign_coords({'nEns':[140,170], 'time':np.arange(1,121)})
z

## reorder dimensions
# z.transpose('time','nEns','xt_ocean','yt_ocean')

<xarray.Dataset>
Dimensions:   (nEns: 2, time: 120, xt_ocean: 360, yt_ocean: 200)
Coordinates:
  * time      (time) object 0170-01-16 12:00:00 ... 0179-12-16 12:00:00
  * xt_ocean  (xt_ocean) float64 -279.5 -278.5 -277.5 -276.5 ... 77.5 78.5 79.5
  * yt_ocean  (yt_ocean) float64 -81.5 -80.5 -79.5 -78.5 ... 86.5 87.5 88.5 89.5
Dimensions without coordinates: nEns
Data variables:
    sst       (nEns, time, yt_ocean, xt_ocean) float32 nan nan nan ... nan nan
Attributes:
    filename:                  01700101.ocean_month.nc
    title:                     ESM2M_pi-control_topaz_C3_verona_weddell0170_n...
    grid_type:                 mosaic
    grid_tile:                 1
    history:                   Sun Dec 22 15:08:36 2019: ncks -4 -L 1 -a -O d...
    NCO:                       netCDF Operators version 4.7.5 (Homepage = htt...
    nco_openmp_thread_number:  1

In [107]:
list(reg_masks.data_vars)

['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']

In [11]:
reg_masks = xr.open_dataset(writedir+'regional_global_masks.nc')
regions = list(reg_masks.data_vars)

ocean_grid = xr.open_dataset(rootdir+'GRID/ocean.static.nc')
geolat_t = ocean_grid.geolat_t
geolon_t = ocean_grid.geolon_t

time = np.arange(1,121)

start_yrs = ['0170_0179', '0022_0031', '0064_0073', '0106_0115', '0232_0241', '0295_0304']

rootdir = '/local/projects/so_predict/esm2m_froelicher/'
writedir = '/home/bbuchovecky/storage/so_predict_derived/'

# var = 'sst'
# subdir = var.upper()+'_ENSEMBLE/'

variables = np.array([['SST','sst','sst'], ['SSS','sss','sss'], ['MLD','mld','mld'], ['NPP','NPP','NPP'],
                      ['PCO2SURF','pco2surf','pco2surf'], ['CN','CN_inv','CN_inv'], ['HI','HI_ice','HI']])

for var in variables:
    yrs_list = []
    
    ## iterate through the start years
    for (i,yrs) in zip(range(1,7), start_yrs):
        ens_list = []
        
        ## iterate through ensembles
        for sign in ['neg', 'pos']:
            for pert in range(1,21):
                subdir = var[0]
                filename = var[1]+'_ENS'+f'{i:02}'+'_'+sign+f'{pert:02}'+'_'+yrs+'.nc'

                ds = xr.open_dataset(rootdir+subdir+filename)

                area = xr.where(np.isnan(ds[var[2]][0]), np.nan, ocean_grid['area_t'])
                    
                ## reassign coord names to ensure continuity between datasets
                coords = list(ds.coords)
                if (coords.count('XT') == 1) and (coords.count('YT') == 1) and (coords.count('TIME') == 1):
                    ds = ds.rename({'XT':'xt_ocean', 'YT':'yt_ocean', 'TIME':'time'})

                if (coords.count('XT_OCEAN') == 1) and (coords.count('YT_OCEAN') == 1) and (coords.count('TIME') == 1):
                    ds = ds.rename({'XT_OCEAN':'xt_ocean', 'YT_OCEAN':'yt_ocean', 'TIME':'time'})
                
                ## assign ocean_grid coords for continuity
                ## convert time from cftime.DatetimeNoLeap to float (1-120)
                ds = ds.assign_coords({'xt_ocean':ocean_grid.xt_ocean, 'yt_ocean':ocean_grid.yt_ocean, 'time':time})
                
                ## compute timeseries for each region
                regs_list = []
                for reg in regions:
                    
                
                
                ens_list.append(ds)
        
        these_ens = xr.concat(ens_list, 'nEns')
        start_list.append(these_ens)
    
    all_ens = xr.concat(yrs_list, 'nStart')
    
    filename = writedir+var[1].upper()+'/'+var[1].lower()+'_ens_'
    all_ens.to_netcdf()

    
    
## compute timeseries for each region
## using ocean_grid coordinates (area_t) and reg_masks

## compute anomaly by subtracting climatology
## using method with NumPy arrays

## save these timeseries in a matrix of dimension (nT,nStart,nEns), 
## where nT is the number of time samples (nT=12 month/year*10 years=120), 
## nStart is the number of start years (6), 
## and nEns is the number of ensemble members (40)

In [28]:
for pert in range(1,21):
    print(f'{pert:02}')

01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20


In [21]:
for (ens,yrs) in zip(range(1,7), ens_range):
    print(str(ens)+' '+yrs)

1 0170_0179
2 0022_0031
3 0064_0073
4 0106_0115
5 0232_0241
6 0295_0304
